In [1]:
import pandas as pd
import numpy as np

# 数据处理

- 转换文件格式，生成csv文件

In [2]:
fp_origin = open("./Analysis.txt", 'r')
fp_modified = open("./Analysis.csv", 'w')

In [3]:
line = fp_origin.readline()
while(line):
    temp = line.strip().split()
    temp = ','.join(temp)+'\n'
    fp_modified.write(temp)
    line = fp_origin.readline()
fp_origin.close()
fp_modified.close()

# 数据分析要求

**1. 数据可视化和摘要**

- 数据摘要

对标称属性，给出每个可能取值的频数，
数值属性，给出最大、最小、均值、中位数、四分位数及缺失值的个数。

- 数据的可视化

针对数值属性：

绘制直方图，如mxPH，用qq图检验其分布是否为正态分布。
绘制盒图，对离群值进行识别
对7种海藻，分别绘制其数量与标称变量，如size的条件盒图

**2. 数据缺失的处理**

- 分别使用下列四种策略对缺失值进行处理:

将缺失部分剔除
用最高频率值来填补缺失值
通过属性的相关关系来填补缺失值
通过数据对象之间的相似性来填补缺失值
处理后，可视化地对比新旧数据集。

** Step1.**
- 读取csv文件，生成data frame

In [4]:
data = pd.read_csv("./Analysis.csv", 
                   names = ["season", "river_size", "river_speed", 
                            "mxPH", "mnO2", "Cl", "NO3", "NH4", "oPO4", "PO4", "Chla", 
                            "a1", "a2", "a3", "a4", "a5", "a6", "a7"], 
                   na_values = "XXXXXXX")

In [5]:
data

,season,river_size,river_speed,mxPH,mnO2,Cl,NO3,NH4,oPO4,PO4,Chla,a1,a2,a3,a4,a5,a6,a7
0,winter,small,medium,8.00,9.80,60.80000,6.238,578.00000,105.00000,170.00000,50.000,0.0,0.0,0.0,0.0,34.2,8.3,0.0
1,spring,small,medium,8.35,8.00,57.75000,1.288,370.00000,428.75000,558.75000,1.300,1.4,7.6,4.8,1.9,6.7,0.0,2.1
2,autumn,small,medium,8.10,11.40,40.02000,5.330,346.66699,125.66700,187.05701,15.600,3.3,53.6,1.9,0.0,0.0,0.0,9.7
3,spring,small,medium,8.07,4.80,77.36400,2.302,98.18200,61.18200,138.70000,1.400,3.1,41.0,18.9,0.0,1.4,0.0,1.4
4,autumn,small,medium,8.06,9.00,55.35000,10.416,233.70000,58.22200,97.58000,10.500,9.2,2.9,7.5,0.0,7.5,4.1,1.0
5,winter,small,high,8.25,13.10,65.75000,9.248,430.00000,18.25000,56.66700,28.400,15.1,14.6,1.4,0.0,22.5,12.6,2.9
6,summer,small,high,8.15,10.30,73.25000,1.535,110.00000,61.25000,111.75000,3.200,2.4,1.2,3.2,3.9,5.8,6.8,0.0
7,autumn,small,high,8.05,10.60,59.06700,4.990,205.66701,44.66700,77.43400,6.900,18.2,1.6,0.0,0.0,5.5,8.7,0.0
8,winter,small,medium,8.70,3.40,21.95000,0.886,102.75000,36.30000,71.00000,5.544,25.4,5.4,2.5,0.0,0.0,0.0,0.0
9,winter,small,high,7.93,9.90,8.00000,1.390,5.80000,27.25000,46.60000,0.800,17.0,0.0,0.0,2.9,0.0,0.0,1.7


In [6]:
data.describe()

,mxPH,mnO2,Cl,NO3,NH4,oPO4,PO4,Chla,a1,a2,a3,a4,a5,a6,a7
count,199.000000,198.000000,190.000000,198.000000,198.000000,198.000000,198.000000,188.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000
mean,8.011734,9.117778,43.636279,3.282389,501.295828,73.590596,137.882101,13.971197,16.923500,7.458500,4.309500,1.992500,5.064500,5.96400,2.495500
std,0.598305,2.391253,46.831311,3.776474,1962.545461,91.136436,128.993738,20.495920,21.348376,11.028202,6.948537,4.417404,7.491401,11.66071,5.158564
min,5.600000,1.500000,0.222000,0.050000,5.000000,1.000000,1.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,7.700000,7.725000,10.981250,1.296000,38.333250,15.700000,41.375250,2.000000,1.500000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,8.060000,9.800000,32.730000,2.675000,103.166500,40.150000,103.285500,5.475000,6.950000,3.000000,1.550000,0.000000,1.900000,0.00000,1.000000
75%,8.400000,10.800000,57.823500,4.446250,226.950007,99.333250,213.750000,18.307500,24.800000,11.375000,4.925000,2.400000,7.500000,6.92500,2.400000
max,9.700000,13.400000,391.500000,45.650000,24064.000000,564.599980,771.599980,110.456000,89.800000,72.600000,42.800000,44.600000,44.400000,77.60000,31.600000
